#### Step 3 – Compute attention metrics during meditation - Breath, Stop (MW), Self

1. Computes meditation stats in each condition for each subject - subj/step3_1_conditionStats.csv. Reads "step2_event_clean_all.txt" to count separate events of the same type (file includes 0,1,3,4)
    - Percentage time
    - **Events**:
        - Number
        - Mean Duration
        - Mean SD

2. Compiles stats from all subjs for SPSS - step3_compile/step3_1_3cat_conditionStats.csv
3. To compare analyses: Compiles average stats for each analysis - step3_compile/1_step3_average_conditionStats.csv ¶

In [2]:
import os
import numpy as np
import scipy.io as sio
import pandas as pd

# set root directory
root_dir = "/Path/To/EMBODY_Study/Files"

# Enter analyses
analysis = "phase1_demo"

# Enter subj list
subjects = [124]

# Enter subj group
group_step2 = ['control']

### Define Functions

In [3]:
def read_txt_file(filename):
    """Read meditation file, returns array of digits

        filename: input is string (e.g., "step2_3cat.txt")

    """

    with open(filename, 'rw') as f:
        content = f.readlines()
    content = list(content[0])

    # filter items in content by whether or not they are a digit. if digit, convert to integer.
    content = [int(x) for x in content if x.isdigit()] 

    return content

In [4]:
# Condition percentage
# stores in column based on condition number, so needs at least 4 columns
def percents():

    decision_count=[0, 0, 0, 0, 0]
    percent=[0, 0, 0, 0, 0]
    conditions=[1, 2, 3, 4, 5]
    
    for condition in conditions:
    
        for decision in meditation: 
            if decision == condition:
                decision_count[condition - 1] += 1
    
    dec_percent = [100 * (x / float(sum(med_length))) for x in decision_count]
    
    return dec_percent

In [5]:
# Number of events per condition, skips 0s
def events(): 
    
    events_count=[0,0,0,0,0]
    
    for index in range(len(meditation)):
        
        current_value = meditation[index]
        previous_value = meditation[index-1]
        
        if index == 0 and current_value != 0:        
            events_count[current_value-1] += 1
        elif current_value == 0:
            continue
        elif current_value != previous_value:
            events_count[current_value-1] += 1
    
    return events_count

In [6]:
# List of durations for each event
def durationsList():
    
    durations = [[0], [0], [0], [0], [0]]
    
    # Counts durations for every element except last
    for index in range(len(meditation)-1):
        
        current_value = meditation[index]
        next_value = meditation[index+1]
        
        if current_value == 0:
            continue
        elif current_value != next_value:
            durations[current_value-1][-1] += 1 
            durations[current_value-1].append(0)
        elif current_value == next_value:
            durations[current_value-1][-1] += 1
    
    # Counts last element
    if current_value != 0:
        durations[meditation[-1]-1][-1] += 1
    
    return durations

In [7]:
# Mean duration
def meanDuration():
    
    durations = durationsList()
    duration_mean = [np.mean([el for el in sublist if el > 0]) for sublist in durations]
    
    return duration_mean

In [8]:
# Standard deviation of duration
def standardDeviation():
    
    durations = durationsList()
    stdev = [np.std([el for el in sublist if el > 0], ddof=1) for sublist in durations]
    
    return stdev

In [16]:
# concatenate pct duration mean and stdev into columns
def compile_stats():
    
    percent = percents()
    event_count = events()
    mean = meanDuration()
    stdev = standardDeviation()
    
    meditation_stats = [percent[0], percent[2], percent[3], 
                        event_count[0], event_count[2], event_count[3],
                        mean[0], mean[2], mean[3],
                        stdev[0], stdev[2], stdev[3]]
    
    index_list = ['pct_breath', 'pct_stop', 'pct_self', 
                  'events_breath', 'events_stop', 'events_self', 
                  'mean_breath', 'mean_stop', 'mean_self', 
                  'sd_breath', 'sd_stop', 'sd_self']
    
    meditation_stats_df = pd.DataFrame(meditation_stats, index = index_list, columns = [subj]) # convert to pandas df
    meditation_stats_df_ts = pd.DataFrame.transpose(meditation_stats_df) # transpose to 1 row
    meditation_stats_df_ts.to_csv('step3_1_3cat_conditionStats.csv') # writes out data

In [17]:
### 1. Computes meditation stats in each condition for each subject

for subj in subjects:
        
    os.chdir('%s/%s/results/step2/%d/' % (root_dir, analysis, subj))

    infile = "step2_event_clean.txt"
    includefile = "step2_include.txt"

    meditation = read_txt_file(infile)
    med_length = read_txt_file(includefile)

    # write out data
    compile_stats()

In [20]:
## 2. Compile step3 stats across subjects - for SPSS

average_stats_compile = []

    
appended_data = []

for subj in subjects:
    os.chdir('%s/%s/results/step2/%d/' % (root_dir, analysis, subj))
    data = pd.read_csv('step3_1_3cat_conditionStats.csv', sep=',', index_col=0)
    # store DataFrame in list
    appended_data.append(data)

# Concatenate meditation stats
df = pd.concat(appended_data, axis=0) # concat by rows
df.insert(0, "group", group_step2) # insert group in first column after index

# write DataFrame to csv 
os.chdir('%s/%s/results/step3_compile/' % (root_dir, analysis))
df.to_csv('step3_1_3cat_conditionStats.csv')

In [23]:
## 3. Compute mean stats and compile across analyses

med_stats_mean = df.mean(axis=0) # mean of all columns
med_stats_mean_df = pd.DataFrame(med_stats_mean, columns = [analysis]) # mean of all columns
med_stats_mean_ts = pd.DataFrame.transpose(med_stats_mean_df) # transpose to 1 row
average_stats_compile.append(med_stats_mean_ts)

# save analysis compilation to csv
df_avg = pd.DataFrame(average_stats_compile[0])
os.chdir('%s/%s/results/analysis_stats/step3_compile/' % (root_dir, analysis))
df_avg.to_csv("step3_1_average_med_stats.csv")